# Exercise 3

### Developing an application for examining the format of the films and,if necessary, convert them.

#### Install FFmpeg library

As we already know, we have to download a static build of FFmpeg to the Coursera server, unzip it, and add the directory to the environment variable “PATH”.

In [9]:
# Download latest FFmpeg static build.  
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/bin/ffmpeg


In [21]:
import os
import sys
import subprocess
import re
import datetime

print()
print('*' * 46)
print('**   Video Format Checker And Re-Encoder    **')
print('*' * 46)
print()

src_dir = 'Exercise3_Films/'
out_dir = 'converted_Films/'

if not os.path.exists(src_dir):
    print('Specified source directory ({}) not exists. Program exited.'.format(src_dir))
    sys.exit()
elif not os.path.exists(out_dir):
    print('Specified output directory ({}) not exists. Program exited.'.format(out_dir))
    sys.exit()

log_data = []

def print_and_log(text):
    log_data.append(text)
    print(text)

for src_vid in os.listdir(src_dir):
    print_and_log('Checking: {}'.format(src_vid))
    vid_exif_call = 'ffprobe -loglevel error -i "{}/{}" -show_streams'.format(src_dir, src_vid)
    exif = subprocess.check_output(vid_exif_call, shell=True).decode('utf-8')
    streams = re.findall('\[STREAM\](?:.|\n)+?\[/STREAM\]', exif)
    vid_exif, aud_exif = [eve for eve in streams if 'codec_type=video' in eve][0], [eve for eve in streams if 'codec_type=audio' in eve][0]

    vid_ext = '.' + src_vid.rsplit('.', 1)[1].lower()

    vid_codec = re.search('(?<=codec_name=)[A-Z-a-z0-9]+', vid_exif)
    vid_codec = vid_exif[vid_codec.start():vid_codec.end()]
    
    aud_codec = re.search('(?<=codec_name=)[A-Z-a-z0-9]+', aud_exif)
    aud_codec = aud_exif[aud_codec.start():aud_codec.end()]

    vid_fps = re.search('(?<=r_frame_rate=)[A-Z-a-z0-9/]+', vid_exif)
    vid_fps = int(round(eval(vid_exif[vid_fps.start():vid_fps.end()]), 1))

    aspect_ratio = re.search('(?<=display_aspect_ratio=)[A-Za-z0-9:]+', vid_exif)
    aspect_ratio = vid_exif[aspect_ratio.start():aspect_ratio.end()]

    vid_width = re.search('(?<=width=)\d+', vid_exif)
    vid_width = vid_exif[vid_width.start():vid_width.end()]

    vid_height = re.search('(?<=height=)\d+', vid_exif)
    vid_height = vid_exif[vid_height.start():vid_height.end()]
    vid_res = '{}x{}'.format(vid_width, vid_height)

    vid_bitrate = re.search('(?<=bit_rate=)\d+', vid_exif)
    vid_bitrate = int(vid_exif[vid_bitrate.start():vid_bitrate.end()])

    aud_bitrate = re.search('(?<=bit_rate=)\d+', aud_exif)
    aud_bitrate = int(aud_exif[aud_bitrate.start():aud_bitrate.end()])

    aud_channels = re.search('(?<=channel_layout=)[A-Z-a-z0-9]+', aud_exif)
    aud_channels = aud_exif[aud_channels.start():aud_channels.end()]

    re_encoding_need = False

    if vid_ext != '.mp4':
        print_and_log('\tVideo file extension invalid: .mp4 vs {}'.format(vid_ext))
        re_encoding_need = True
        
    if vid_codec != 'h264':
        print_and_log('\tVideo codec invalid: h264 vs {}'.format(vid_codec))
        re_encoding_need = True
        
    if aud_codec != 'aac':
        print_and_log('\tAudio codec invalid: aac vs {}'.format(aud_codec))
        re_encoding_need = True
        
    if vid_fps != 25:
        print_and_log('\tVideo FPS invalid: 25 vs {}'.format(vid_fps))
        re_encoding_need = True
        
    if aspect_ratio != '16:9':
        print_and_log('\tAspect ratio invalid: 16:9 vs {}'.format(aspect_ratio))
        re_encoding_need = True
        
    if vid_res != '640x360':
        print_and_log('\tVideo resolution invalid: 640x360 vs {}'.format(vid_res))
        re_encoding_need = True
        
    if not 2000000 <= vid_bitrate <= 5000000:
        print_and_log('\tVideo bitrate invalid: {:.2f} Mb/s is not in range of 2 - 5 Mb/s'.format(float(vid_bitrate / 1000000)))
        re_encoding_need = True
        
    if not aud_bitrate <= 256000:
        print_and_log('\tAudio bitrate invalid: {} Kb/s is higher than 256 Kb/s'.format(int(aud_bitrate / 1000)))
        re_encoding_need = True
        
    if aud_channels != 'stereo':
        print_and_log('\tAudio channels invalid: stereo vs {}'.format(aud_channels))
        re_encoding_need = True

    if re_encoding_need:
        print_and_log('\tRe-encoding...')
        re_encode_com = 'ffmpeg -y -loglevel error -i "{}/{}" -aspect 16:9 -filter:v fps=fps=25 -s 640x360 -ac 2 -codec:v h264 -codec:a aac -b:v 5M -b:a 256k "{}/{}.mp4"'.format(src_dir, src_vid, out_dir, src_vid.replace('.', '_formatOK.').rsplit('.', 1)[0])
        subprocess.call(re_encode_com, shell=True)
    else:
        print_and_log('\tFormat matched, no need to re-encode!')

    print_and_log('')

log_filename = datetime.datetime.now().strftime('report_%d%m%Y.txt')
log_file_op = open(out_dir + '\\' + log_filename, encoding='utf-8', mode='w')
log_file_op.write('\n'.join(log_data))
log_file_op.close()


**********************************************
**   Video Format Checker And Re-Encoder    **
**********************************************

Checking: Cosmos_War_of_the_Planets.mp4
	Video FPS invalid: 25 vs 30
	Aspect ratio invalid: 16:9 vs 314:177
	Video resolution invalid: 640x360 vs 628x354
	Audio bitrate invalid: 317 Kb/s is higher than 256 Kb/s
	Re-encoding...

Checking: Last_man_on_earth_1964.mov
	Video file extension invalid: .mp4 vs .mov
	Video codec invalid: h264 vs prores
	Audio codec invalid: aac vs pcm
	Video FPS invalid: 25 vs 24
	Video bitrate invalid: 9.29 Mb/s is not in range of 2 - 5 Mb/s
	Audio bitrate invalid: 1536 Kb/s is higher than 256 Kb/s
	Re-encoding...

Checking: The_Hill_Gang_Rides_Again.mp4
	Video bitrate invalid: 7.54 Mb/s is not in range of 2 - 5 Mb/s
	Re-encoding...

Checking: Voyage_to_the_Planet_of_Prehistoric_Women.mp4
	Video codec invalid: h264 vs hevc
	Audio codec invalid: aac vs mp3
	Video FPS invalid: 25 vs 30
	Video bitrate invalid: 8.04 Mb/s is

### Output

#### The_Gun_and_the_Pulpit

In [18]:
from IPython.display import Video
Video('converted_Films/The_Gun_and_the_Pulpit_formatOK.mp4')

#### Cosmos_War_of_the_Planets

In [23]:
Video('converted_Films/Cosmos_War_of_the_Planets_formatOK.mp4')

#### Last_man_on_earth_1964

In [24]:
Video('converted_Films/Last_man_on_earth_1964_formatOK.mp4')

#### The_Hill_Gang_Rides_Again

In [25]:
Video('converted_Films/The_Hill_Gang_Rides_Again_formatOK.mp4')

#### Voyage_to_the_Planet_of_Prehistoric_Women

In [26]:
Video('converted_Films/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4')